In [ ]:
import datetime
import numpy as np
import pandas as pd
import wordcloud as wc
import nltk

# Ejemplo de Twitter: ver [vínculo](https://towardsdatascience.com/tweepy-for-beginners-24baf21f2c25)

In [ ]:
import tweepy

class TweetMiner(object):

    result_limit    =   20    
    data            =   []
    api             =   False

    twitter_keys = {
        'consumer_key':        '---YOUR-KEY---',
        'consumer_secret':     '---YOUR-KEY---',
        'access_token_key':    '---YOUR-KEY---',
        'access_token_secret': '---YOUR-KEY---'
    }
    
    tweet_mode = 'extended'

    def __init__(self, keys_dict=twitter_keys, api=api, result_limit = 20):
        self.twitter_keys = keys_dict
        auth = tweepy.OAuthHandler(keys_dict['consumer_key'], keys_dict['consumer_secret'])
        auth.set_access_token(keys_dict['access_token_key'], keys_dict['access_token_secret'])
        self.api = tweepy.API(auth)
        self.twitter_keys = keys_dict
        self.result_limit = result_limit
        

    def mine_user_tweets(
        self, user="nytimes",
        mine_rewteets=False,
        max_pages=20
    ):
        data           =  []
        last_tweet_id  =  False
        page           =  1
        while page <= max_pages:
            if last_tweet_id:
                statuses   =   self.api.user_timeline(screen_name=user,
                    count=self.result_limit,
                    max_id=last_tweet_id - 1,
                    tweet_mode = 'extended',
                    include_retweets=mine_rewteets
                )        
            else:
                statuses   =   self.api.user_timeline(screen_name=user,
                    count=self.result_limit,
                    tweet_mode = 'extended',
                    include_retweets=mine_rewteets
                )
                
            for item in statuses:
                mined = {
                    'tweet_id':        item.id,
                    'name':            item.user.name,
                    'screen_name':     item.user.screen_name,
                    'retweet_count':   item.retweet_count,
                    'text':            item.full_text,
                    'mined_at':        datetime.datetime.now(),
                    'created_at':      item.created_at,
                    'favourite_count': item.favorite_count,
                    'hashtags':        item.entities['hashtags'],
                    'status_count':    item.user.statuses_count,
                    'location':        item.place,
                    'source_device':   item.source
                }
                try:
                    mined['retweet_text'] = item.retweeted_status.full_text
                except:
                    mined['retweet_text'] = 'None'
                try:
                    mined['quote_text'] = item.quoted_status.full_text
                    mined['quote_screen_name'] = status.quoted_status.user.screen_name
                except:
                    mined['quote_text'] = 'None'
                    mined['quote_screen_name'] = 'None'
                last_tweet_id = item.id
                data.append(mined)
            page += 1
            
        return data

In [ ]:
miner=TweetMiner(result_limit = 200)
mined_tweets = miner.mine_user_tweets(user='casaleantonio', max_pages=20)
mined_tweets_df= pd.DataFrame(mined_tweets)

In [ ]:
mined_tweets_df.text

**Crear un vector de palabras únicas:**

Pueden usar ```set(['p1','p2','p3',...])```

<mark>Nota 1:</mark> pueden hacer un prefiltrado usando lo que se conoce como **stopwords** (más información acceda [aquí](https://www.ranks.nl/stopwords)) y la herramienta para hacerlo es [NLTK (Natural Language ToolKit)](https://github.com/nltk/nltk/tree/develop/nltk). Aquí les dejo el [link!](https://stackoverflow.com/questions/5541745/get-rid-of-stopwords-and-punctuation)

<mark>Nota 2:</mark> para los histogramas usen ```plt.hist``` o ```sns.distplot``` el que sea de su preferencia. Algo interesante es aprender a usar las nubes de palabras. Aquí les dejo un [ejemplo interesante de GOT](https://kaparker.com/posts/creating-word-clouds-with-python) ([otra opción más sofisticada](https://www.datacamp.com/community/tutorials/wordcloud-python)) y la [¡librería!](https://github.com/amueller/word_cloud)

# Ejemplo de FIFA

Se descarga un archivo comprimizo y se puede descomprimir como `unzip fifa-20-complete-player-dataset.zip` que tiene los siguientes contenidos
```bash
players_15.csv
players_16.csv
players_17.csv
players_18.csv
players_19.csv
players_20.csv
```
Cada archivo de texto tiene el mismo número de columnas y contiene información interesante. Los invito a que lo abran (Microsoft Excel, LibreOffice, o el de su preferencia) Si no quieren hacerlo, pueden mirar las columnas con el siguiente comando: e.g. `head -n1 players15.csv | sed 's/,/\n/g'` con el siguiente output:
```bash
sofifa_id
player_url
short_name
long_name
age
dob
height_cm
weight_kg
nationality
club
overall
potential
value_eur
wage_eur
player_positions
preferred_foot
international_reputation
weak_foot
skill_moves
work_rate
body_type
real_face
release_clause_eur
player_tags
team_position
team_jersey_number
loaned_from
joined
contract_valid_until
nation_position
nation_jersey_number
pace
shooting
passing
dribbling
defending
physic
gk_diving
gk_handling
gk_kicking
gk_reflexes
gk_speed
gk_positioning
player_traits
attacking_crossing
attacking_finishing
attacking_heading_accuracy
attacking_short_passing
attacking_volleys
skill_dribbling
skill_curve
skill_fk_accuracy
skill_long_passing
skill_ball_control
movement_acceleration
movement_sprint_speed
movement_agility
movement_reactions
movement_balance
power_shot_power
power_jumping
power_stamina
power_strength
power_long_shots
mentality_aggression
mentality_interceptions
mentality_positioning
mentality_vision
mentality_penalties
mentality_composure
defending_marking
defending_standing_tackle
defending_sliding_tackle
goalkeeping_diving
goalkeeping_handling
goalkeeping_kicking
goalkeeping_positioning
goalkeeping_reflexes
ls
st
rs
lw
lf
cf
rf
rw
lam
cam
ram
lm
lcm
cm
rcm
rm
lwb
ldm
cdm
rdm
rwb
lb
lcb
cb
rcb
rb
```

Para el ejercicio nos interesan: <u>nationality</u> y <u>overall</u> (¿quizás <u>potential</u>?)
Para cargar el archivo, por ejemplo `players_15.csv` y sólo esas columnas podemos usar el comando de Numpy [loadtxt](https://docs.scipy.org/doc/numpy/reference/generated/numpy.loadtxt.html) o [genfromtxt](https://docs.scipy.org/doc/numpy/reference/generated/numpy.genfromtxt.html). Aquí les dejo el dos scripts de ejemplo para cada comando:
```python
test1 = np.loadtxt(
    'players_15.csv', skiprows=1, usecols=(8,10,11), delimiter=',',
    dtype={
        'names': ('nationality', 'overall', 'potential'),
        'formats': ('S20', np.int8, np.int8)
    }
)
print(test1)

test2 = np.genfromtxt(
    'players_15.csv', skip_header=True, usecols=(8,10,11), delimiter=',',
    dtype=None, names=('nationality', 'overall', 'potential'),
    encoding=None
)
print(test2)
```